In [18]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [19]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [20]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [4]:
dict_table['Group B']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
915,Belgium,Canada,2022,1,0,1
916,Switzerland,Cameroon,2022,1,0,1
917,Uruguay,South Korea,2022,0,0,0
918,Portugal,Ghana,2022,3,2,5


In [5]:
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [6]:
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [7]:
df_team_strength = pd.concat([df_home,df_away],ignore_index=True) .groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.682927,1.158537
Australia,0.823529,2.058824
Austria,1.482759,1.620690
...,...,...
Uruguay,1.526316,1.298246
Wales,0.714286,1.000000
West Germany,2.112903,1.241935


In [8]:
def predict_points(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11):
            for y in range(0,11):
                p=poisson.pmf(x,lamb_home) * poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw += p
                elif x>y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home,points_away)
    else:
        return (0,0)

In [9]:
predict_points('Qatar (H)','Ecuador')

(0.2560433697238933, 2.6268524842459953)

In [10]:
predict_points('Germany','Japan')

(2.3365943407623795, 0.510188578181496)

In [11]:
predict_points('Argentina','Saudi Arabia')

(2.72905933634538, 0.19159383151813886)

# Group Stage

In [79]:
up_table={'Group A':    [['Pos',"Team"  ,'Pld'  ,'W'  ,'D' , 'L' , 'GF' , 'GA',  'GD' , 'Pts'],
    [1 ,'Qatar (H)'    ,0 , 0  ,0 , 0 ,  0,   0  , 0  ,  0],
  [ 2   ,   'Ecuador'  ,  0  ,0  ,0 , 0   ,0   ,0 ,  0  ,  0],
   [3    ,  'Senegal'  ,  0,  0 , 0,  0 ,  0 ,  0  , 0 ,   0],
    [4  ,'Netherlands'  ,  0 , 0  ,0,  0  , 0 ,  0  , 0 ,   0]]}


In [68]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [80]:
up_table=pd.DataFrame(up_table['Group A'])


In [83]:
up_table = up_table.set_index(['Team'])

KeyError: "None of ['Team'] are in the columns"

In [93]:
up_table['Group A']

KeyError: 'Group A'

In [92]:
#up_table[up_table.columns[0]]
#print(up_table.iloc[0])
up_table['Group A'].set_index(up_table.iloc[0].values)

KeyError: 'Group A'

In [61]:
type(dict_table)

dict

In [ ]:
"""
 'Group B':    Pos           Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1        England    0  0  0  0   0   0   0    0
 1    2           Iran    0  0  0  0   0   0   0    0
 2    3  United States    0  0  0  0   0   0   0    0
 3    4          Wales    0  0  0  0   0   0   0    0,
 'Group C':    Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Argentina    0  0  0  0   0   0   0    0
 1    2  Saudi Arabia    0  0  0  0   0   0   0    0
 2    3        Mexico    0  0  0  0   0   0   0    0
 3    4        Poland    0  0  0  0   0   0   0    0,
 'Group D':    Pos       Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     France    0  0  0  0   0   0   0    0
 1    2  Australia    0  0  0  0   0   0   0    0
 2    3    Denmark    0  0  0  0   0   0   0    0
 3    4    Tunisia    0  0  0  0   0   0   0    0,
 'Group E':    Pos        Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1       Spain    0  0  0  0   0   0   0    0
 1    2  Costa Rica    0  0  0  0   0   0   0    0
 2    3     Germany    0  0  0  0   0   0   0    0
 3    4       Japan    0  0  0  0   0   0   0    0,
 'Group F':    Pos     Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1  Belgium    0  0  0  0   0   0   0    0
 1    2   Canada    0  0  0  0   0   0   0    0
 2    3  Morocco    0  0  0  0   0   0   0    0
 3    4  Croatia    0  0  0  0   0   0   0    0,
 'Group G':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1       Brazil    0  0  0  0   0   0   0    0
 1    2       Serbia    0  0  0  0   0   0   0    0
 2    3  Switzerland    0  0  0  0   0   0   0    0
 3    4     Cameroon    0  0  0  0   0   0   0    0,
 'Group H':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Portugal    0  0  0  0   0   0   0    0
 1    2        Ghana    0  0  0  0   0   0   0    0
 2    3      Uruguay    0  0  0  0   0   0   0    0
 3    4  South Korea    0  0  0  0   0   0   0    0"""

In [12]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [16]:
df_fixture_group_48.keys()

Index(['home', 'score', 'away', 'year'], dtype='object')

In [14]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home,away = row['home'],row['away']
        points_home,points_away=predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] ==home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] ==away, 'Pts'] += points_away
        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team','Pts']]
    dict_table[group] = dict_table[group].round(0)

In [67]:
for group in up_table:
    teams_in_group = up_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home,away = row['home'],row['away']
        points_home,points_away=predict_points(home,away)
        up_table[group].loc[up_table[group]['Team'] ==home, 'Pts'] += points_home
        up_table[group].loc[up_table[group]['Team'] ==away, 'Pts'] += points_away
        
    up_table[group] = up_table[group].sort_values('Pts', ascending=False).reset_index()
    up_table[group] = up_table[group][['Team','Pts']]
    up_table[group] = up_table[group].round(0)

TypeError: list indices must be integers or slices, not str

In [15]:
dict_table['Group F']

,Team,Pts
0,Croatia,7.0
1,Belgium,6.0
2,Morocco,4.0
3,Canada,1.0


# knockout